# How to build an AI Agent with PydanticAI and the ClickHouse MCP Server

In this notebook we'll see how to build an [PydanticAI](https://ai.pydantic.dev/mcp/client/#__tabbed_1_1) AI agent that can interact with [ClickHouse's SQL playground](https://sql.clickhouse.com/) using [ClickHouse's MCP Server](https://github.com/ClickHouse/mcp-clickhouse).


## Install libraries
We need to install the Agno library.

In [ ]:
!pip install -q --upgrade pip

In [38]:
!pip install -q "pydantic-ai-slim[mcp]"
!pip install -q "pydantic-ai-slim[anthropic]"

## Setup credentials
Let's provide our Anthropic API key.

In [2]:
import os, getpass

In [3]:
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter Anthropic API Key:")

Enter Anthropic API Key: ········


We'll also define the credentials to connect to the ClickHouse SQL playground:

In [ ]:
env = {
    "CLICKHOUSE_HOST": "sql-clickhouse.clickhouse.com",
    "CLICKHOUSE_PORT": "8443",
    "CLICKHOUSE_USER": "demo",
    "CLICKHOUSE_PASSWORD": "",
    "CLICKHOUSE_SECURE": "true"
}

## Initialize MCP Server and PydanticAI agent

Lets configure the ClickHouse MCP Server to point at the ClickHouse SQL playground and also initialize our PydanticAI agent and ask it a question:

In [5]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerStdio
from pydantic_ai.messages import ToolCallPart, ToolReturnPart

In [6]:
server = MCPServerStdio(  
    'uv',
    args=[
        'run',
        '--with', 'mcp-clickhouse',
        '--python', '3.13',
        'mcp-clickhouse'
    ],
    env=env
)
agent = Agent('anthropic:claude-sonnet-4-0', mcp_servers=[server])

## Ask the agent a question

And now let's ask the agent a question:

In [45]:
async with agent.run_mcp_servers():
    result = await agent.run("Who's done the most PRs for ClickHouse?")
    print(result.output)

Based on the data from the ClickHouse GitHub repository, here are the top contributors by number of pull requests created:

**Top contributors to ClickHouse by PRs opened:**

1. **alexey-milovidov** - 3,370 PRs opened
2. **azat** - 1,905 PRs opened  
3. **rschu1ze** - 979 PRs opened
4. **alesapin** - 947 PRs opened
5. **tavplubix** - 896 PRs opened
6. **kssenii** - 871 PRs opened
7. **Avogar** - 805 PRs opened
8. **KochetovNicolai** - 700 PRs opened
9. **Algunenano** - 658 PRs opened
10. **kitaisreal** - 630 PRs opened

**Alexey Milovidov** stands out as by far the most active contributor with over 3,370 pull requests opened, which is significantly more than any other contributor. This makes sense as Alexey Milovidov is one of the founders and lead developers of ClickHouse.

The data also shows that alexey-milovidov has been very active in managing PRs, with 12,818 "closed" events (likely reviewing and closing PRs from other contributors) in addition to creating his own PRs.

It's wort